# Cultural Data Analysis

Introduction to working with datasets

In [ ]:
# import necessary libraries
import os, re, csv
import pandas as pd
import numpy as np

## Loading the dataset: heritage homes webistes

The dataset is stored in a shared google drive:
https://drive.google.com/drive/folders/11Shm0edDOiWrOe56fzJQRZi-v_BPSW8E?usp=drive_link

Add it to your drive.

To access it, load your gdrive in 'Files' (see left pane of the notebook in google colab) and navigate to the shared folder. You may need to click on 'refresh' to make it appear on the list.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/jazoza/cultural-data-analysis


### Import all datasets (4 countries)

You will have all datasets available for analysis and comparison, mapped in the following way:

> df0 - Dutch dataset

> df1 - UK dataset

> df2 - German dataset

> df3 - French dataset

In [ ]:
# Country code: change here between 'NL' and 'UK'
cc_list = ['NL', 'UK', 'DE', 'FR']

In [ ]:
gdrive_path = '/content/gdrive/MyDrive/CDA/'

In [ ]:
# Import scraped json data into 4 separate dataframes
df0=pd.read_json(gdrive_path+cc_list[0]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df0=df0[['url','text','metadata']]

df1=pd.read_json(gdrive_path+cc_list[1]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df1=df1[['url','text','metadata']]

df2=pd.read_json(gdrive_path+cc_list[2]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df2=df2[['url','text','metadata']]

df3=pd.read_json(gdrive_path+cc_list[3]+'_dataset_website-content-crawler.json')
# select columns for analysis: url, text, metadata
df3=df3[['url','text','metadata']]

df0.head()

In [ ]:
# function to extract the main domain from the url in the dataset
def extract_main_domain(url):
    if not isinstance(str(url), str):
        print('NOT VALID',url)
        return None
    match = re.findall('(?:\\w+\\.)*\\w+\\.\\w*', str(url)) #'www\.?([^/]+)'
    return match[0].lstrip('www.') if match else None

In [ ]:
# Add a new column 'domain' and fill it by applying the extract_main_domain function to the 'url' column

# first, create a mapping of dataframes which could be addressed in a loop
df_dict = {'0':df0, '1':df1, '2':df2, '3':df3}

# then, loop through the df_dict to update each dataframe
for k, v in df_dict.items():
  cc_column = cc_list[int(k[-1])]+' domains'
  cc = cc_list[int(k[-1])]
  # print(cc_column, cc)
  urls = pd.read_csv(gdrive_path+'url_lists/'+cc_list[int(k[-1])]+'_urls.csv')[cc_column].values.tolist()
  domains = {extract_main_domain(url) for url in urls if extract_main_domain(url) is not None}
  matching_links = [link for link in v.url if extract_main_domain(link) in domains]
  # update the dataframe
  v['domain'] = v['url'].apply(extract_main_domain)

# check one of the dataframes
df1.head()

### Import stopwords

Import stopwords dictionaries for the 4 langauges we work with. It is good to import all of them in our case, because many websites have sections is English, German or French even when this is not the main language of the website.

In [ ]:
# load a list of 'stopwords' function
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [ ]:
# Get the stopwords list for all languages (using cc_list previously defined)
# cc_list = ['NL', 'UK', 'DE', 'FR'] # remove the hashtag from this line to uncomment this code and make it run

stopwords = [] # empty list to which a list of stopwords will be appended in loop

for i in range(len(cc_list)):
  stopwords_cc_path = "/content/cultural-data-analysis/stopwords_archive/"+cc_list[i]+".txt"
  stopwords_cc = get_stopwords_list(stopwords_cc_path)
  #print(len(stopwords_cc)) # print how many words are in the list
  stopwords.extend(stopwords_cc)

#print(len(stopwords)) # print how many words are in all stopwords lists

In [ ]:
# you may need to include additional words which you notice as too frequent
special_stop_words = ['nbsp', 'nl', 'fr', 'de', 'uk', 'com', 'www', 'lit', ' '] # these might appear frequently as 'terms' in the corpus, so it's good to filter them
stopwords_ext = stopwords+special_stop_words

## 1. Visualize term frequency: bar chart

In [ ]:
# CALCULATE TERM FREQUENCY WITHOUT STOP-WORDS
from sklearn.feature_extraction.text import CountVectorizer

#cvec_stopped = CountVectorizer(max_df=0.5, token_pattern=r'(?u)\b[A-Za-z]{2,}\b') # max_df could in theory automatically filter stopwords
cvec_stopped = CountVectorizer(stop_words=stopwords_ext, token_pattern=r'(?u)\b[A-Za-z]{2,}\b') # token pattern recognizes only words which are made of letters, and longer than 1 character
cvec_stopped.fit(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
document_matrix = cvec_stopped.transform(df0.text) #### CHANGE df0 TO THE DATAFRAME YOU ANALYSE
term_batches = np.linspace(0,document_matrix.shape[0],10).astype(int)
i=0
df_stopped = []
while i < len(term_batches)-1:
    batch_result = np.sum(document_matrix[term_batches[i]:term_batches[i+1]].toarray(),axis=0)
    df_stopped.append(batch_result)
    print(term_batches[i+1],"entries' term frequency calculated")
    i += 1

terms_stopped = np.sum(df_stopped,axis=0)
#print(terms_stopped.shape)
term_freq_df_stopped = pd.DataFrame([terms_stopped],columns=cvec_stopped.get_feature_names_out()).transpose()
term_freq_df_stopped.columns = ['terms']

In [ ]:
import matplotlib.pyplot as plt

# Filter and order top N words in descending order
# change the value in .head(N) to include more or less terms
topN = term_freq_df_stopped.sort_values(by='terms',
                                                ascending=False).head(50)

# Create the bar chart
plt.figure(figsize=(12, 6))
plt.bar(topN.index, topN['terms'], color='skyblue')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Most Frequent Words (excluding stopwords)')
plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

# Create the bar chart using seaborn
plt.figure(figsize=(12, 6))
sns.barplot(x=topN.index, y=topN['terms'], palette='viridis',
            hue=topN.index, legend=False)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 10 Most Frequent Words (excluding stopwords)')
plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

### 1.1 Visualize comparative frequency

Compare the frequency of female- and male-connotated titles

In [ ]:
# define lists of nobility titles for each county
# lists are defined as tupples: value pairs of title and 0/1
# where 0 = male titles, 1 = female title
dutch_nobility_titles = [('ridder',0), ('jonkvrouw',1),
                        ('baron',0), ('barones',1),
                         ('graaf',0), ('gravin',1),
                          ('hertog',0), ('hertogin',1),
                          ('prins',0), ('prinses',1)]

uk_nobility_titles = [('sir',0), ('lady',1), ('knight',0),
                        ('baron',0), ('baroness',1),
                         ('duke',0), ('duchess',1),
                          ('prince',0), ('princess',1),
                           ('king',0), ('queen',1)]

german_nobility_titles = [ ('ritter',0),
                         ('graf',0), ('gräfin',1),
                          ('fürst',0),('fürstin',1),
                          ('herzog',0), ('herzogin',1),
                           ('prinz',0), ('prinzessin',1),
                            ('könig', 0),('königin', 1)]

french_nobility_titles = [('chevalier',0), ('dame',1), ('baron',0), ('baronne',1),
                          ('duc',0), ('duchesse',1), ('marquis',0), ('marquise',1),
                           ('prince',0), ('princesse',1), ('roi',0), ('reine',1)]


In [ ]:
# change country for nobility titles
# the list of titles should match the country dataframe
# df0 - dutch_nobility_titles
# df1 - uk_nobility_titles
# df2 - german_nobility_titles
# df3 - french_nobility_titles

nobility_titles = german_nobility_titles
df = df2

nobility = [term for term, gender in nobility_titles]
genders = [gender for term, gender in nobility_titles]

for term in nobility:
    df[term] = df['text'].apply(lambda x: x.lower().count(term) if isinstance(x, str) else 0)

# Filter for rows where 'kasteel' appears at least once
mask = (df[nobility] > 0).any(axis=1)

# Filter the DataFrame based on the boolean mask.
df_filtered = df[mask]

# Create the five-column table
filter_list = ['domain','url'] + nobility
result_df = df_filtered[filter_list]

# Calculate the sum of values for each specified column
column_sums = result_df[nobility].sum()

# Create a bar chart with pink bars
fig, ax = plt.subplots(figsize=(10,6))
colors = ['#e877f0' if gender == 0 else '#9f01aa' for gender in genders]

# Add horizontal lines at 1/8ths of the vertical bars height
max_bar_height = column_sums.values.max()
step_size = max_bar_height / 8
# Ensure at least one line is drawn if max_bar_height is very small
if step_size == 0 and max_bar_height > 0:
    y_values = [max_bar_height]
elif step_size > 0:
    y_values = np.arange(step_size, max_bar_height + step_size, step_size)
else:
    y_values = []

for value in y_values:
  plt.axhline(y=value, color='grey', linestyle='--', linewidth=0.5)

plt.bar(column_sums.index, column_sums.values, color=colors)

# Customize the plot
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

ax.set_ylabel('Sum of term frequencies')
ax.set_title('Frequency of terms related to nobility titles in 2024 corpus heritage houses websites')

# Add two columns from the df dataframe on the left
plt.text(-0.8, -160, f"total websites: {len(df.domain.unique())}", fontsize=10)
plt.show()

## 2. Visualize term relations: scatter plot



In [ ]:
!pip install gensim

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import gensim
from nltk.tokenize import word_tokenize

df = df0

# X is a list of tokenized texts (i.e. list of lists of tokens)
X = [word_tokenize(item) for item in df.text.tolist()] # replace df0 with a dataframe you are analysing
#print(X[0:3])
model = gensim.models.Word2Vec(X, min_count=6, vector_size=200) # min_count: how many times a word appears in the corpus; size: number of dimensions



### Visualize keywords with t-SNE

Visualize to N number, or choose keywords that correspond to your analysis and visualize how they and their closest terms are distributed in the discourse.
Use t-SNE to visualize the relations.

* [t-Distributed Stochastic Neighbor Embedding (t-SNE)](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets.


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.cm as cm
import seaborn as sns

# Get the top N words from term_freq_df_stopped
topN_df = term_freq_df_stopped.sort_values(by='terms', ascending=False).head(1000)
topN_words_list = topN_df.index.tolist()

# Filter words that are present in the Word2Vec model's vocabulary
filtered_top_words = [word for word in topN_words_list if word in model.wv.key_to_index]

# Get the word embeddings for these filtered words
embeddings_for_tsne = [model.wv[word] for word in filtered_top_words]

# Check if there are embeddings to visualize
if not embeddings_for_tsne:
    print("No valid words found in the model's vocabulary from the top 100 words to visualize.")
else:
    embeddings_array = np.array(embeddings_for_tsne)

    # Apply t-SNE for Dimensionality Reduction
    tsne_model_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32)
    embeddings_2d = tsne_model_2d.fit_transform(embeddings_array)

    # Use seaborn style for plots
    sns.set_style("whitegrid")

    # Visualize the 2D word embeddings
    plt.figure(figsize=(16, 9))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, c='blue')

    for i, word in enumerate(filtered_top_words):
        plt.annotate(word, alpha=0.8, xy=(embeddings_2d[i, 0], embeddings_2d[i, 1]),
                     xytext=(5, 2), textcoords='offset points', ha='right', va='bottom', size=8)

    plt.title('t-SNE Visualization of Top 100 Most Frequent Words')
    plt.xlabel('t-SNE Dimension 1')
    plt.ylabel('t-SNE Dimension 2')
    plt.grid(True)
    plt.show()

### 2.1 Visualise relations between specific words

In [ ]:
search_terms = ['man', 'vrouw']

In [ ]:
here


## 4. Topic Modeling


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

documents = df['text'].tolist()

vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words=stopwords_ext) # Modify stopwords if needed
dtm = vectorizer.fit_transform(documents)

num_topics = 10
lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
lda.fit(dtm)


no_top_words = 10
no_top_documents = 10

display_topics(lda.components_, lda.transform(dtm), vectorizer.get_feature_names_out(), documents, no_top_words, no_top_documents)

#### Visualize topics

## 5. Visualize word collocation